# Only Flask Server

In [ ]:
from twilio.rest import Client

account_sid = 'fin₫ your account sid'
auth_token = 'find your auth token'
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='whatsapp:+14155238886',
  body='Your appointment is coming up on July 21 at 3PM',
  to='whatsapp:+91xxxxxxxxxx'
)

print(message.sid)

In [2]:
from flask import Flask, request, jsonify
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import numpy as np
import pandas as pd
import joblib
from flask_cors import CORS

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_classifier = loaded_pipeline.named_steps['svc']

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

app = Flask(__name__)
CORS(app)

limiter = Limiter(
    get_remote_address,
    app=app,
    default_limits=["200 per day", "5 per hour"],
    storage_uri="memory://",
)

@app.route('/predict', methods=['POST'])
@limiter.limit("5 per minute")
def predict():
    data = request.get_json()
    x_test = data['features']
    received_array = np.array(x_test)
    columns_to_select = ['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2',
       'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'age', 'ethnicity', 'gender']
    received_dataframe = pd.DataFrame(data=[received_array], columns=columns_to_select)
    X_new = received_dataframe
    X_new_standardized = scaler.transform(X_new)
    prediction = svm_classifier.predict(X_new_standardized).astype(int)[0]

    confidence_scores = svm_classifier.decision_function(X_new_standardized)
    confidence_percentage = round(float(sigmoid(confidence_scores) * 100),2)

    return jsonify({'prediction': prediction.tolist(), 'confidence': confidence_percentage})


if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2024 21:00:40] "POST /predict HTTP/1.1" 200 -
